# Check what you are voting for with PyTezos

In [1]:
import sys
assert sys.version_info.major == 3 and sys.version_info.minor >= 6

In [2]:
import os
sys.path.insert(1, os.path.abspath('..'))

### Determine current voting phase

In [3]:
from pytezos.rpc import mainnet

In [4]:
mainnet.head.votes.current_period_kind()

'proposal'

### Get current period bounds

In [5]:
level_info = mainnet.head.metadata.get('level')
level_info

{'level': 348350,
 'level_position': 348349,
 'cycle': 85,
 'cycle_position': 189,
 'voting_period': 10,
 'voting_period_position': 20669,
 'expected_commitment': False}

In [6]:
head, tail = level_info['level_position'], level_info['level_position'] - level_info['voting_period_position']
head, tail

(348349, 327680)

### Check what are the current proposals

In [7]:
proposals = mainnet.head.votes.proposals()
proposals

[['Psd1ynUBhMZAeajwcZJAeq5NrxorM6UCU4GJqxZ7Bx2e9vUWB6z', 5176],
 ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd', 11744]]

Let's examine one of proposals

In [8]:
proposal_id = proposals[1][0]
proposal_id

'Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd'

### Search for the injection operation

In [ ]:
from pytezos.rpc.search import SearchChain

In [ ]:
sc = SearchChain.from_chain(mainnet.main)

Thanks to the statefullness of the Tezos blockchain we can perform a binary search inside the voting period we defined earlier

In [ ]:
operation = sc.find_proposal_inject_operation(proposal_id)

In [ ]:
operation()

We can also verify operation signature

In [ ]:
operation.verify_signature()

The only thing we now about the submitter's identity is public key

In [ ]:
mainnet.get_public_key(operation.source())

### Get source code of the current protocol

In [ ]:
current_proto = mainnet.protocols[operation.get('protocol')]
current_proto

Load files from blockchain, encode (without protocol-compiler binary) and get hash

In [ ]:
current_hash = current_proto.calculate_hash()
current_hash

In [ ]:
assert current_hash == operation.get('protocol')

### Get source code of the proposed update

Unfortunately it's practically impossible to get proposal source from the blockchain.
We could get lucky and find the submitter's node (in case it's public) which has to know this proto (according to the docs).
The other option is try to find a node in zeronet which has participated in the voting rehearsal.

In [9]:
tar_url = 'https://blog.nomadic-labs.com/files/Athens_proposal_A.tar'

In [10]:
from pytezos.rpc.protocol import Protocol

Loading sources and convert them to the internal format (as in blockchain)

In [ ]:
athens_a = Protocol.from_uri(tar_url)

1176680it [03:20, 2343.60it/s]

In [ ]:
proto_id = athens_a.calculate_hash()
proto_id

In [ ]:
assert proposal_id == proto_id

### Get unified diff

In [ ]:
patch = athens_a.diff(current_proto, context_lines=3)

In [ ]:
from pytezos.tools.diff import generate_html

This tool can generate github-like side-by-side patch views, powered by diff2html.js

In [ ]:
generate_html(patch, output_path='report.html')

[View report](./report.html)

### Get all voting operations for this proposal

In [ ]:
for operation in sc.find_proposal_votes_operations(proposals[0][0]):
    print(operation())
    break  # this can take a while

Search algorithm works as follows:
1. Split block interval into equal chunks
2. Determine which of the intervals contain state changes
3. For each interval run binary search
4. If there are several changes inside single interval, run binary search again

It's obvious that the search space can be easily splitted and processed independently, i.e parallelized. 

### Storing protocol diff

In [ ]:
ctxless_patch = athens_a.diff(current_proto, context_lines=0)

In [ ]:
ctxless_patch.export_tar('diff.tar.bz2')

In [ ]:
os.path.getsize('diff.tar.bz2')

As we pointed earlier there is no convenient way to get proposal source from the blockchain. This can be implemented via smart-contract. But it's more reasonable to store compressed code diff rather than full source.

Compare 13 KB vs 1.2 MB (original tar archive)

### Applying protocol diff

In [ ]:
proto = current_proto.apply(ctxless_patch)

In [ ]:
assert proto_id == proto.calculate_hash()